In [1]:
import sys
sys.path.append('./Tools/')
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import random
import google
import pandas as pd

In [2]:
 # Use the application default credentials
cred = credentials.Certificate('./Tools/polibrain-24d47-firebase-adminsdk-406ve-1a87461e28.json')
firebase_admin.initialize_app(cred, {
  'projectId': 'polibrain-24d47',
})

db = firestore.client()

In [3]:
# Pegando o desempenho por questao dos usuarios
emailListT2 = []
camadaListT2 = []
nivelListT2 = []
id_questaoList2 = []
acertoListT2 = []
tempoListT2 = []

docs = db.collection(u'usuarios_pont_questoes').get()
for doc in docs:
    emailListT2.append(doc.to_dict()['email_usr'])
    camadaListT2.append(doc.to_dict()['camada'])
    nivelListT2.append(doc.to_dict()['nivel'])
    id_questaoList2.append(doc.to_dict()['id_questao'])
    acertoListT2.append(doc.to_dict()['acerto'])
    tempoListT2.append(doc.to_dict()['tempo'])

In [4]:
df_usr_questoes = pd.DataFrame(data = {'email_usr':emailListT2, 'camada':camadaListT2, 'nivel':nivelListT2, 'id_questao':id_questaoList2, 'acerto':acertoListT2, 'tempo':tempoListT2})

In [5]:
df_clustering = df_usr_questoes.groupby(['camada','nivel','id_questao']).mean()[['acerto','tempo']].reset_index()

In [6]:
df_clustering.head()

,camada,nivel,id_questao,acerto,tempo
0,1,1,002,1.0,7.0
1,1,1,004,1.0,9.0
2,1,1,007,1.0,4.0
3,1,1,008,1.0,3.0
4,1,1,009,1.0,6.0


In [7]:
from sklearn import cluster
from sklearn import preprocessing
import numpy as np
import pdb

num_clusters = 10


df_clustering_total = pd.DataFrame(data = {})
for cam in df_clustering['camada'].unique():
    for niv in df_clustering['nivel'].unique():
        try:
            df_clustering_util = df_clustering[(df_clustering['camada']==cam) & (df_clustering['nivel']==niv)]
            df_questoes_scaled = pd.DataFrame(data = preprocessing.scale(df_clustering_util[['acerto', 'tempo']]), columns = ['acerto', 'tempo'])
            kmeans = cluster.KMeans(n_clusters=num_clusters).fit(df_questoes_scaled)
            df_questoes_scaled['grupo'] = kmeans.predict(df_questoes_scaled).tolist()
            df_questoes_scaled['camada'] = df_clustering_util['camada'].tolist()
            df_questoes_scaled['nivel'] = df_clustering_util['nivel'].tolist()
            df_questoes_scaled['id_questao'] = df_clustering_util['id_questao'].tolist()
            
            df_aux = pd.DataFrame(data = kmeans.cluster_centers_, columns = ['ca','ct'])
            df_aux['pont'] = (3*df_aux['ca'] - df_aux['ct'])/4
            df_aux['grupo'] = np.arange(0,num_clusters,1).tolist()
            df_aux = df_aux.sort_values(by = 'pont', ascending=False)
            df_aux['grupo_novo'] = np.arange(1,num_clusters+1,1).tolist()
            df_aux = df_aux[['grupo','grupo_novo']]
            df_questoes_scaled = df_questoes_scaled.merge(df_aux, on = 'grupo', how = 'inner')
            
            df_clustering_total = pd.concat([df_clustering_total, df_questoes_scaled])
        except:
            print("Problema na camada " + str(cam) + " e no nível " + str(niv))

Problema na camada 1 e no nível 3
Problema na camada 2 e no nível 5
Problema na camada 2 e no nível 6
Problema na camada 3 e no nível 2
Problema na camada 3 e no nível 4
Problema na camada 3 e no nível 5
Problema na camada 3 e no nível 6
Problema na camada 3 e no nível 3


/home/alan/.local/lib/python2.7/site-packages/sklearn/cluster/k_means_.py:968: ConvergenceWarning: Number of distinct clusters (9) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  return_n_iter=True)


In [8]:
df_clustering_total = df_clustering_total[['camada','nivel','id_questao','grupo_novo']]

In [9]:
df_clustering_total.head()

,camada,nivel,id_questao,grupo_novo
0,1,1,002,2
1,1,1,009,2
2,1,1,019,2
3,1,1,033,2
4,1,1,041,2


In [10]:
camadas_dict = {1:'mat_bas',2:'fracoes',3:'equacoes'}
niveis_dict = {(1,1):'soma',(1,2):'subtracao',(1,3):'expressoes1',(1,4):'expressoes2',(1,5):'multiplicacao',(1,6):'divisao',
              (2,1):'mdc',(2,2):'mmc',(2,3):'irredutiveis',(2,4):'somafrac',(2,5):'multfrac',(2,6):'divfrac',
              (3,1):'1grau'}

In [11]:
df_clustering_total['camada'].tolist()[0]

1

In [12]:
for i in range (0,df_clustering_total.shape[0]):
    c = df_clustering_total['camada'].tolist()[i]
    n = df_clustering_total['nivel'].tolist()[i]
    idq = df_clustering_total['id_questao'].tolist()[i]
    g = df_clustering_total['grupo_novo'].tolist()[i]
    questoes_ref = db.collection('questoes_mat').document(camadas_dict[c]).collection(niveis_dict[c,n]).document(idq)
    questoes_ref.update({u'grupo': g})